# European Football Statistics

In [17]:
import pandas as pd
import requests
import http.client
import json
from pandas.io.json import json_normalize

key = '80c2f8ba62244830a2eda578176435d9'

In [14]:
#Import Excel file with Club Market Values

excel = 'C:/Users/USER/Documents/Springboard/Capstone 1/club_values.xlsx'
sheets = [426, 430, 434, 436, 438]

columns = ["Clubs", "Unnamed: 1", "Squad", "Age", "Foreign players", "Total market value", "Market value"]
club_values = pd.DataFrame(columns=columns)
for i in sheets:
    df = pd.read_excel(open(excel,'rb'), sheetname= str(i))
    df = df.iloc[1:,:]
    df.columns = ["Clubs", "Unnamed: 1", "Squad", "Age", "Foreign players", "Total market value", "Market value"]
    club_values =club_values.append(df, ignore_index=True)

#Standardize Club names   

club_values.loc[24,"Clubs"]="Red Bull Leipzig"
club_values.loc[20,"Clubs"]="FC Bayern München"
club_values.loc[25,"Clubs"]="Bor. Mönchengladbach"
club_values.loc[22,"Clubs"]="Bayer Leverkusen"
club_values.loc[30,"Clubs"]="Werder Bremen"
club_values.loc[32,"Clubs"]="1. FSV Mainz "
club_values.loc[42,"Clubs"]="Olympique de Marseille"
club_values.loc[46,"Clubs"]="OSC Lille"
club_values.loc[49,"Clubs"]="Montpellier Hérault SC"
club_values.loc[44,"Clubs"]="FC Girondins de Bordeaux"
club_values.loc[56,"Clubs"]="Dijon FCO"
club_values.loc[45,"Clubs"]="Toulouse FC"
club_values.loc[57,"Clubs"]="AS Nancy"
club_values.loc[51,"Clubs"]="Angers SCO"
club_values.loc[64,"Clubs"]="Athletic Club"
club_values.loc[75,"Clubs"]="CD Leganes"
club_values.loc[60,"Clubs"]="Club Atlético de Madrid"
club_values.loc[70,"Clubs"]="Real Betis"
club_values.loc[71,"Clubs"]="RCD Espanyol"
club_values.loc[73,"Clubs"]="RC Deportivo La Coruna"
club_values.loc[97,"Clubs"]="US Cittá di Palermo"
club_values.loc[78,"Clubs"]="Juventus Turin"
club_values.loc[91,"Clubs"]="Bologna FC"
club_values.loc[94,"Clubs"]="Pescara Calcio"
club_values.loc[93,"Clubs"]="Empoli FC"
club_values.loc[81,"Clubs"]="FC Internazionale Milano"

In [19]:
#Load League Data From: http://api.football-data.org

connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': key, 'X-Response-Control': 'minified' }
connection.request('GET', '/v1/competitions/?season=2016', None, headers )
response = json.loads(connection.getresponse().read().decode())

#Create data frame of top 5 most important leagues

top_leagues_df = pd.DataFrame(response)
top_leagues_df = top_leagues_df[top_leagues_df['league'].isin(['BL1', 'FL1', 'PL', 'PD', 'SA'])]
top_leagues_df.rename(columns={'caption':'name'}, inplace=True)
top_leagues_df = top_leagues_df.reset_index()
top_leagues_df = top_leagues_df[['id', 'name', 'league', 'numberOfTeams']]
top_leagues_df['name'] = top_leagues_df['name'].map(lambda x: str(x)[:-8])
top_leagues_df.columns = ['league_id', 'league_name', 'league', 'number_of_eams']

countries = ['United Kingdom', 'Germany', 'France', 'Spain', 'Italy']
top_leagues_df['country'] = countries

In [21]:
#Load Team Data From: http://api.football-data.org

league_list = top_leagues_df['league_id']
columns = ['crestUrl', 'id', 'name', 'shortName', 'squadMarketValue', 'league_id']
teams_df = pd.DataFrame(columns=columns)
teams_df
id = 0
for i in league_list:
    connection = http.client.HTTPConnection('api.football-data.org')
    headers = { 'X-Auth-Token': key, 'X-Response-Control': 'minified' }
    connection.request('GET', '/v1/competitions/'+str(i)+'/teams', None, headers )
    response = json.loads(connection.getresponse().read().decode())
    data_df = pd.DataFrame(response)
    data_df = data_df['teams']
    teams_list=[]
    for i in data_df:
        teams_list.append(i)
    data_df = pd.DataFrame.from_dict(teams_list)
    data_df['league_id'] = league_list[id]
    teams_df = teams_df.append(data_df, ignore_index=True)
    id+=1
teams_df = teams_df.sort_values('id')

In [33]:
#loads teams' standing at end of 2016/2017 season from: http://api.football-data.org

columns = ["Home Draws", "Home Goals", "Home Goals Against", "Home Losses", "Home Wins"]
standings_df = pd.DataFrame(columns=columns)
league_list = list(top_leagues_df['league_id'])
id = 0
for i in league_list:
    url='http://api.football-data.org/v1/competitions/'+str(i)+'/leagueTable/?apikey=80c2f8ba62244830a2eda578176435d9'
    r = requests.get(url)
    json_data = r.json()
    json_data = json_data['standing']
    for i in json_data:
        data={}
        for k, v in i.items():
            if k not in ['home', 'away']:
                data[k] = v
        data_df =pd.DataFrame.from_dict(data)
        data_df = data_df.reset_index(drop=True)
        venue={}
        for k, v in i.items():
            if k in ['home', 'away']:
                venue[k] = v
        df =pd.DataFrame.from_dict(venue)
        df =df.transpose()
        away_df =pd.DataFrame(df.iloc[0,:])
        away_df =away_df.transpose()
        away_df.columns = ["Away Draws", "Away Goals", "Away Goals Against", "Away Losses", "Away Wins"]
        away_df = away_df.reset_index(drop=True)
        home_df =pd.DataFrame(df.iloc[1,:])
        home_df =home_df.transpose()
        home_df.columns = ["Home Draws", "Home Goals", "Home Goals Against", "Home Losses", "Home Wins"]
        home_df = home_df.reset_index(drop=True)
        venue_df = pd.concat([home_df, away_df], axis=1)
        team_stats_df = pd.concat([data_df, venue_df], axis=1)
        team_stats_df = team_stats_df.set_index('_links')
        team_stats_df['league_id'] = league_list[id]
        standings_df = standings_df.append(team_stats_df, ignore_index=True)
    id += 1

In [25]:
# Load Player Data From: http://api.football-data.org

#Creates Data Frame for half of players

columns = ['id', 'name', 'dateOfBirth', 'position', 'nationality', 'marketValue', 'jerseyNumber', 'contractUntil']
players_df1 = pd.DataFrame(columns=columns)
teams_list = list(teams_df['id'])
teams_list = teams_list[0:49]
id = 0
for i in teams_list:
    connection = http.client.HTTPConnection('api.football-data.org')
    headers = { 'X-Auth-Token': key, 'X-Response-Control': 'minified' }
    connection.request('GET', '/v1/teams/'+str(i)+'/players', None, headers )
    response = json.loads(connection.getresponse().read().decode())     
    data_df = pd.DataFrame(response)
    data_df = data_df['players']
    players_list=[]
    for i in data_df:
        players_list.append(i)
    data_df = pd.DataFrame.from_dict(players_list)
    data_df['team_id'] = teams_list[id]
    players_df1 = players_df1.append(data_df, ignore_index=True)
    id+=1

In [27]:
#Creates Data Frame for second half of players    
    
columns = ['id', 'name', 'dateOfBirth', 'position', 'nationality', 'marketValue', 'jerseyNumber', 'contractUntil']
players_df2 = pd.DataFrame(columns=columns)
teams_list = list(teams_df['id'])
teams_list = teams_list[49:]
id = 0
for i in teams_list:
    connection = http.client.HTTPConnection('api.football-data.org')
    headers = { 'X-Auth-Token': key, 'X-Response-Control': 'minified' }
    connection.request('GET', '/v1/teams/'+str(i)+'/players', None, headers )
    response = json.loads(connection.getresponse().read().decode())     
    data_df = pd.DataFrame(response)
    data_df = data_df['players']
    players_list=[]
    for i in data_df:
        players_list.append(i)
    data_df = pd.DataFrame.from_dict(players_list)
    data_df['team_id'] = teams_list[id]
    players_df2 = players_df2.append(data_df, ignore_index=True)
    id+=1

In [28]:
#Merge players data frames into one
players_df = players_df1.append(players_df2, ignore_index=True)

In [31]:
#Renames, reorders and changes columns data types for leagues data frame

Top_leagues_df = top_leagues_df[['league_id', 'league_name', 'number_of_eams', 'country']]
Top_leagues_df.loc[:,'league_id'], Top_leagues_df.loc[:,'number_of_eams'] = Top_leagues_df.loc[:,'league_id'].astype(int), Top_leagues_df.loc[:,'number_of_eams'].astype(int)
Top_leagues_df.columns = ["League ID", "League Name", "Number of Teams", "Country"]

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [44]:
#Renames, reorders and changes columns data types for teams data frame

Teams_df = teams_df[['id', 'name', 'squadMarketValue', 'league_id']]
Teams_df.loc[:,'id'], Teams_df.loc[:,'league_id'] = Teams_df.loc[:,'id'].astype(int), Teams_df.loc[:,'league_id'].astype(int)
Teams_df.columns = ["Team ID", "Team Name", "Team Value Mill. €", "League ID"]
Teams_df = pd.merge(Teams_df, Top_leagues_df, on= "League ID")
Teams_df = Teams_df.drop("Number of Teams", 1)
Teams_df = Teams_df.sort_values("Team Name")
Teams_df = Teams_df.sort_values("League Name")
Teams_df =Teams_df.reset_index(drop=True)

#Adds market value from club values data frame

for x in club_values.itertuples():
     for i in Teams_df.itertuples():
        if x[1] in i[2]:
            Teams_df.set_value(i[0], "Team Value Mill. €", x[6])

Teams_df['Team Value Mill. €'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
Teams_df[['Team Value Mill. €']] = Teams_df[['Team Value Mill. €']].astype(int)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [37]:
#Renames, reorders standings data frame

Standings_df = standings_df[['position','teamName', 'wins', 'draws', 'losses', 'goals',
                             'goalsAgainst', 'goalDifference', 'points', 'Home Wins', 'Home Losses',
                             'Home Draws', 'Home Goals', 'Home Goals Against', 'Away Wins',
                             'Away Draws', 'Away Losses','Away Goals', 'Away Goals Against',
                             'crestURI']]
Standings_df.columns=  ["Position","Team Name", "Wins", "Draws", "Losses", "Goals",
                        "Goals Against", "Goal Difference", "Points", "Home Wins", "Home Losses",
                        "Home Draws", "Home Goals", "Home Goals Against", "Away Wins",
                        "Away Draws", "Away Losses","Away Goals", "Away Goals Against",
                         "Crest"]

In [39]:
#Renames, reorders players data frame
Players_df = players_df[['id', 'name', 'position', 'marketValue', 'dateOfBirth',
                         'contractUntil', 'nationality', 'team_id', 'jerseyNumber']]
Players_df.loc[:,'id'], Players_df.loc[:,'team_id'] = Players_df.loc[:,'id'].astype(int), Players_df.loc[:,'team_id'].astype(int)
Players_df.columns = ["Player Id", "Name", "Position", "Market Value", "Date of Birth",
                      "Contract Until", "Nationality", "Team ID", "Jersey Number"]
Players_df = pd.merge(Players_df, Teams_df, on= "Team ID")
Players_df = Players_df.drop(["Country", "League ID", "Team Value Mill. €"], 1)
Players_df = Players_df.sort_values("Name")
Players_df = Players_df.sort_values("League Name")

In [59]:
#Creates Team Stat Summary

Summary_df = pd.merge(Standings_df, Teams_df, on= "Team Name")
Summary_df = Summary_df.drop(['Home Wins',
       'Home Losses', 'Home Draws', 'Home Goals', 'Home Goals Against',
       'Away Wins', 'Away Draws', 'Away Losses', 'Away Goals',
       'Away Goals Against', 'Crest', 'Team ID','League ID', 'Country'], 1)

# Output Tables

Top Leagues Table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Shape : 5 x 4



In [63]:
Top_leagues_df.head(5)

,League ID,League Name,Number of Teams,Country
0,426,Premier League,20,United Kingdom
1,430,1. Bundesliga,18,Germany
2,434,Ligue 1,20,France
3,436,Primera Division,20,Spain
4,438,Serie A,20,Italy


.

Teams Table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Shape : 98 x 6


In [64]:
Teams_df.head(5)

,Team ID,Team Name,Team Value Mill. €,League ID,League Name,Country
0,1,1. FC Köln,11285,430,1. Bundesliga,Germany
1,721,Red Bull Leipzig,16338,430,1. Bundesliga,Germany
2,17,SC Freiburg,8185,430,1. Bundesliga,Germany
3,16,FC Augsburg,5873,430,1. Bundesliga,Germany
4,55,SV Darmstadt 98,2040,430,1. Bundesliga,Germany


.

Standings Leagues Table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Shape : 98 x 22

Columns ='Away Draws', 'Away Goals', 'Away Goals Against', 'Away Losses',
       'Away Wins', 'Home Draws', 'Home Goals', 'Home Goals Against',
       'Home Losses', 'Home Wins', 'crestURI', 'draws', 'goalDifference',
       'goals', 'goalsAgainst', 'league_id', 'losses', 'playedGames', 'points',
       'position', 'teamName', 'wins'


In [73]:
Standings_df.head(5)

,Position,Team Name,Wins,Draws,Losses,Goals,Goals Against,Goal Difference,Points,Home Wins,Home Losses,Home Draws,Home Goals,Home Goals Against,Away Wins,Away Draws,Away Losses,Away Goals,Away Goals Against,Crest
0,1.0,Chelsea FC,27.0,3.0,5.0,76.0,32.0,44.0,84.0,15,2,0,47,16,12.0,3.0,3.0,29.0,16.0,http://upload.wikimedia.org/wikipedia/de/5/5c/...
1,2.0,Tottenham Hotspur FC,24.0,8.0,3.0,77.0,23.0,54.0,80.0,16,0,2,45,8,8.0,6.0,3.0,32.0,15.0,http://upload.wikimedia.org/wikipedia/de/b/b4/...
2,3.0,Manchester City FC,20.0,9.0,6.0,68.0,38.0,30.0,69.0,9,1,7,30,16,11.0,2.0,5.0,38.0,22.0,https://upload.wikimedia.org/wikipedia/en/e/eb...
3,4.0,Liverpool FC,20.0,9.0,6.0,71.0,42.0,29.0,69.0,11,2,4,42,18,9.0,5.0,4.0,29.0,24.0,http://upload.wikimedia.org/wikipedia/de/0/0a/...
4,5.0,Arsenal FC,20.0,6.0,9.0,68.0,42.0,26.0,66.0,12,2,3,34,15,8.0,3.0,7.0,34.0,27.0,http://upload.wikimedia.org/wikipedia/en/5/53/...


.

Players Table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Shape : 2465 x 11


In [70]:
Players_df.head(5)

,Player Id,Name,Position,Market Value,Date of Birth,Contract Until,Nationality,Team ID,Jersey Number,Team Name,League Name
419,839,Örjan Nyland,Keeper,None,1990-09-10,2019-06-30,Norway,31,1,FC Ingolstadt 04,1. Bundesliga
418,838,Martin Hansen,Keeper,None,1990-06-15,2020-06-30,Denmark,31,35,FC Ingolstadt 04,1. Bundesliga
229,679,Daniel Didavi,Attacking Midfield,None,1990-02-21,2021-06-30,Germany,11,11,VfL Wolfsburg,1. Bundesliga
426,846,Markus Suttner,Left-Back,None,1987-04-16,2018-06-30,Austria,31,29,FC Ingolstadt 04,1. Bundesliga
445,885,Daniel Heuer Fernandes,Keeper,None,1992-11-13,2018-06-30,Portugal,55,1,SV Darmstadt 98,1. Bundesliga


.

Summary Table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Shape : 98 x 11

In [76]:
Summary_df.head(5)

,Position,Team Name,Wins,Draws,Losses,Goals,Goals Against,Goal Difference,Points,Team Value Mill. €,League Name
0,1.0,Chelsea FC,27.0,3.0,5.0,76.0,32.0,44.0,84.0,55775,Premier League
1,2.0,Tottenham Hotspur FC,24.0,8.0,3.0,77.0,23.0,54.0,80.0,49450,Premier League
2,3.0,Manchester City FC,20.0,9.0,6.0,68.0,38.0,30.0,69.0,54325,Premier League
3,4.0,Liverpool FC,20.0,9.0,6.0,71.0,42.0,29.0,69.0,42220,Premier League
4,5.0,Arsenal FC,20.0,6.0,9.0,68.0,42.0,26.0,66.0,51700,Premier League
